---
format:
  html:
    embed-resources: true
    code-fold: true
---

## Decision Tree and Random Forest

The code used to create my Baseline, a basic Decision Tree, and a Random Forest can be found [here](https://github.com/shawnhxu/EnhancingSBMM/blob/main/codes/07-trees/baseline_and_RF.ipynb).

This was done on the Record data found [here](https://github.com/shawnhxu/EnhancingSBMM/blob/main/data/02-cleaned_riot_data/league_combined_with_chall_cleaned.csv).

```python
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import random
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score,recall_score,f1_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
```

## Methods on Random Forest

With a given dataset of observations and multiple variables, Random Forest (RF) can be done to generalize the data so that results aren't overfitting and has the potential to render noise as less impactful on the final prediction. So, the key idea in RF models is to use the raining dataset to build many, many decision trees where each decision tree are randomly made based on the training dataset. This randomness, again, reinforces the idea of generalization on the final result.<br>

To go into more detail, decision trees are randomly made and what this means is that each decision tree takes a random subset of the training dataset with each node of the tree only considering a random subset of the features. This will create multiple decision trees where each tree will have different nodes with different decisions for the random subset of variables. Later, the test data will run through the model to determine what each random tree predicts. 
<br>

Finally after each tree outputs a prediction, the RF will select a majority voted prediction for the test data sample. And this will be done with all test samples within the test dataset. By introducing randomness and then aggregating the overall predictions to output a final result, it helps to improve the model's ability to properly classify new data using the training data. It will utilize generalization of the data rather than possibly focusing on outliers/noise which can negatively affect the model's predictions.
<br>


## Baseline of Class Distribution

In this step, I took the distribution of the number of counts of the ranks that appeared in the record dataset. This is used later to do a random number generation based on the weights of the distribution which would act as the predictions. This generation of predictions is then compared to a uniformally generated distribution of classes. From here, I calculated the ```accuracy```, ```precision```, ```recall```, and ```F1``` scores and this is what I found: <br>

```
Random Classifier metric scores:
Accuracy Score: 0.1206
Precision Score: 0.12346353020210461
Recall Score: 0.1206
F1 Score: 0.12141405817891071
Precision, Recall, and F1 Scores: (array([0.11662727, 0.10995185, 0.1458671 , 0.13333333, 01288, 0.0811245 , 0.14705882, 0.10090238]), array([0.13143872, 0.11729452, 0.13081395, 0.11925287, 0.11290323, 0.1011011 , 0.12475378, 0.12424242]), array([0.12359081, 0.11350456, 0.13793103, 0.12590064, 0.12032885, 0.09001783, 0.13499112, 0.11136261]), array([1126, 1168, 1376, 1392, 1426,  999, 1523,  990], dtype=int64))
```

As seen in the output ```accuracy```, ```precision```, ```recall```, and ```F1``` scores, it makes sense to see a total accuracy probability of about 0.125 as the distribution of classes for the record data is somewhat uniform meaning the total accuracy is about 1/total number of class or 8 which is about 0.125.Next, we can see that total Precision, Recall, and F1-Score sort of follows the distribution. This is because the scores are also relatively close to 1.125 again though a little off since the weights are not perfectly uniform. To go into more details on each class' precision, recall, and f1 scores the last output shows how each class performed. And we can see that the average precision, recall, and f1 scores are close to the aggregate scores.

The code I used:

In [ ]:
# Simple histogram plot to show the distribution of each rank in the record dataset. 
sns.countplot(x='rank', data = combined_league_ranks)
plt.title('Counts of Each Rank In Record Data')
plt.xticks(rotation = 90)
plt.show()
#the number of samples with given ranks range from ~530 to ~840.


# Now get an actual count of each rank.
rank_dist = combined_league_ranks['rank'].value_counts().reset_index()
rank_dist.columns = ["ranks","counts"]
print(rank_dist)


#add a column of the probability of the distribution of class labels
rank_dist['prob'] = rank_dist['counts']/sum(rank_dist['counts'])
#add another column to convert the ranks into numbers for the random classifier algorithm
label_encode = LabelEncoder()
rank_dist['ranks_num'] = label_encode.fit_transform(rank_dist['ranks'])

# input the numbered classes and the weights of each class into random.choices() to generate 10,000 sampled dataset
rank_weights = random.choices(list(rank_dist['ranks_num']), weights=list(rank_dist['prob']), k = 10000)


#then input into random classifier code
#Code was provided by DSAN 5100 shared code
ypred=[]
max_label=np.max(rank_weights)
for i in range(0,len(rank_weights)):
    ypred.append(int(np.floor((max_label+1)*np.random.uniform(0,1))))

#print accuracy, precision, and recall
print("Random Classifier metric scores:")
print("Accuracy Score:",accuracy_score(rank_weights, ypred))
print("Precision Score:",precision_score(rank_weights, ypred, average='weighted'))
print("Recall Score:",recall_score(rank_weights, ypred, average='weighted'))
print("F1 Score:",f1_score(rank_weights, ypred, average='weighted'))
print("Precision, Recall, and F1 Scores:",precision_recall_fscore_support(rank_weights, ypred))

### Basic Decision Tree Classifier Model

In this step, I decided to use a basic individual Decision Tree in order to create a classification for predicting class labels of test dataset samples. One of the biggest steps to do in making the most optimal Decision Tree model is to find the best hyperparameters for making a Decision Tree. <br> 

To find the best hyperparameters to use for an individual Decision Tree classifier, I found a great function by ```Scikit-Learn``` called ```GridSearchCV``` that takes models and predefined hyperparameter options to go through each combination of the parameters. After going through each combination of hyperparameters, the function can output (at least the output that I specified) an accuracy score of the model on the training dataset. The highest accuracy score is determined and the hyperparameters used to determine this accuracy is outputted as well. After finding the optimal parameters, the Decision Tree found that the highest accuracy it could achieve was about __20.6% accuracy__. This is pretty close with what I found in Naive Bayes. <br>


In [ ]:
# preset some options for hyperparameters then go through the combinations with GridShotCV
param_grid = {'max_depth': [None, 5, 10],'min_samples_split': [2, 5, 10, 20, 40, 80],'min_samples_leaf': [1, 2, 4, 8, 16, 32]}

#split dataset 80/20 = training/testing
x_train, x_test = train_test_split(combined_league_ranks, test_size=0.2, random_state=33)

#pop 'rank' col to make target variables
y_train = x_train.pop("rank")
y_test = x_test.pop("rank")

# now create basic decision tree classifier model
dt_model = DecisionTreeClassifier()

#find best parameters with predefined param_grid
gridsearch = GridSearchCV(dt_model, param_grid, cv=5, scoring='accuracy')
gridsearch.fit(x_train, y_train)

#print the best accuracy found and the tuned hyperparameters to use
print("Optimal Hyperparameters:",gridsearch.best_params_)
print("Training accuracy:",gridsearch.best_score_)

After finding the most optimal hyperparameter combination for the Decision Tree classifier, I fitted it with the testing dataset and got __20.3% accuracy__. With this we can see that the model had about the same accuracy/error with the training and testing datasets. This was the tree that was created: <br>

In [ ]:
#plot the decision tree itself
plt.figure(figsize=(30,15))
plot_tree(opt_dt_model, filled=True, rounded=True, fontsize=12)
plt.savefig('basic_tree.png', bbox_inches='tight')
plt.show()

![Basic Decision Tree](images/basic_tree.png)

And also a confusion matrix on the predictions made by the basic Decision Tree on test data: <br>

In [ ]:
#output the predictions of the basic decision tree classifier
test_preds = opt_dt_model.predict(x_test)
conf = confusion_matrix(y_test, test_preds)
sns.heatmap(conf, annot=True, fmt="d", cmap="Blues", xticklabels= y_train.unique(), yticklabels=y_train.unique())
plt.savefig('basic_tree_conf.png', bbox_inches='tight')
plt.title('Confusion Matrix of a Decision Tree with Optimal Parameters')
plt.show()


![Basic Decision Tree Confusion Matrix](images/basic_tree_conf.png)

The tree itself was pretty full of nodes and leaves so just visually seeing the structure was the best I could achieve in plotting the tree. More importantly, the confusion matrix made seemed pretty similar to the one I made in Naive Bayes tab where the distribution was pretty scattered and certain ranks took most of the predictions. Either way, it definitely showed why the accuracy was low (about 20%). This further emphasizes what I found earlier in my project where the dataset I retrieved was inherently complex and rank predictions might be difficult to achieve. So in the future, I think including more variables might help in determining the best variables to predict players' ranks.

### Random Forest Classifier Model

Finally, I followed the same steps that I did with the basic Decision Tree model but now with a Random Forest model. I similarly used the GridSearchCV function to find the most optimal hyperparameters for the Random Forest model with the training dataset. I then found the accuracy of the training data to be about __23.3%__ with the Random Forest model. This is a slight improvement from what I found with the basic Decision Tree. <br>

In [ ]:
#predefine parameter options
param_grid_rf = {'n_estimators': [50, 100, 150],'max_depth': [None, 5, 10, 15],
    'min_samples_split': [2, 5, 10],'min_samples_leaf': [1, 2, 4]}

#RF model
rf_model = RandomForestClassifier(random_state=33)

#now do gridsearch again on the RF model with diff combos of predefined parameter options
gridsearch_rf = GridSearchCV(rf_model, param_grid_rf, cv=5, scoring='accuracy')
#also fit with train data and targets
gridsearch_rf.fit(x_train, y_train)

print("Optimal hyperparameters:", gridsearch_rf.best_params_)
print("Optimal Model Accuracy: ", gridsearch_rf.best_score_)

Now with the optimal parameters found, I created the Random Forest model again and fitted it with the training data. This time I found a score of about __23.8% accuracy__ which was exciting but overall still a very bad accuracy score for my model. <br>

So making a plot for the Random Forest model is basically impossible as it would require me to plot every single individual tree from the model. I decided to skip this step and simply just plot a confusion matrix of the predicted output: <br>

In [ ]:
#Plot confusion matrix of the new predictions with RF
test_preds_rf = opt_rf.predict(x_test)
conf = confusion_matrix(y_test, test_preds_rf)
sns.heatmap(conf, annot=True, fmt="d", cmap="Blues", xticklabels= y_train.unique(), yticklabels=y_train.unique())
plt.savefig('rf_conf.png', bbox_inches='tight')
plt.title('Confusion Matrix of Random Forest with Optimal Parameters')
plt.show()

![Random Forest Confusion Matrix](images/rf_conf.png)

This confusion matrix looks a bit more spread out instead of focusing on specific ranks. What is consistent is that the 'Challenger' rank is still being correctly predicted massively compared to other ranks. Though, the diagonal of predictions of the ranks is a lot more apparent now. I feel like this confusion matrix illustrates well why the accuracy is slightly higher with Random Forest.

### Conclusions

As stated before, the models used show why my dataset might be too complex to create predictions of player ranks from. Similar to what I found in my Naive Bayes section of my project, I noticed that my model produced an accuracy score of about 20% across the training and test datasets. This  was a low accuracy score and therefore meant that my dataset might be inherently too complex to make a classifier model out of it. <br>

After doing this again with a basic Decision Tree model and a Random Forest model, I noticed the same ~20% accuracy of predicting the correct rank. This further emphasized my assumption that my dataset is inherently too complex and that I might need to include other variables than the ones I chose in order to better predict a player's rank from their unseen match data. So in the future if I were to continue this project, I would definitely look to retrieve different record data and to try these classification models again to see if my scores change at all.